# Payment Channel - BX exercise

In this BX tutorial, we will demonstrate a simple payment channel between two participants, Alice and Bob.

* Wallet Setup for Alice & Bob
* 1) Opening a payment channel
* 2) Updating the payment channel
* 3) Alice broadcasts old state. Bob sweeps the channel.



<hr style="border: 0.5px dashed #000;">


## Wallet setup for Alice & Bob

In [ ]:
# my_mnemonic ="word0 word1 word2 ..."
# hd_master_private=$( )
# hd_master_public=$( )

my_mnemonic="seven mail crash you unit small assume express wedding cloud work potato"
hd_master_private=$(bx mnemonic-to-seed $my_mnemonic | bx hd-new)
hd_master_public=$(bx hd-to-public $hd_master_private)

Alice keys: `m/44'/1'/1'/0/5`

In [ ]:
# hd_m_44h_1h_1h_0_5=

# privatekey_44h_1h_1h_0_5=
# publickey_44h_1h_1h_0_5=
# publickeyhash_44h_1h_1h_0_5=
# address_44h_1h_1h_0_5=

hd_m_44h_1h_1h_0_5=$(bx hd-private --hard --index 44 $hd_master_private \
| bx hd-private --hard --index 1  \
| bx hd-private --hard --index 1  \
| bx hd-private --index 0 \
| bx hd-private --index 5)

privatekey_44h_1h_1h_0_5=$(bx hd-to-ec $hd_m_44h_1h_1h_0_5)
publickey_44h_1h_1h_0_5=$(bx hd-to-public $hd_m_44h_1h_1h_0_5 | bx hd-to-ec)
publickeyhash_44h_1h_1h_0_5=$(bx sha256 $publickey_44h_1h_1h_0_5 | bx ripemd160)
address_44h_1h_1h_0_5=$(bx hd-to-public $hd_m_44h_1h_1h_0_5 | bx hd-to-ec | bx ec-to-address --version 111)

echo $address_44h_1h_1h_0_5
echo $publickey_44h_1h_1h_0_5
echo $publickeyhash_44h_1h_1h_0_5


Bob keys: `m/44'/1'/1'/0/6`

In [ ]:
# hd_m_44h_1h_1h_0_6=

# privatekey_44h_1h_1h_0_6=
# publickey_44h_1h_1h_0_6=
# publickeyhash_44h_1h_1h_0_6=
# address_44h_1h_1h_0_6=

hd_m_44h_1h_1h_0_6=$(bx hd-private --hard --index 44 $hd_master_private \
| bx hd-private --hard --index 1  \
| bx hd-private --hard --index 1  \
| bx hd-private --index 0 \
| bx hd-private --index 6)

privatekey_44h_1h_1h_0_6=$(bx hd-to-ec $hd_m_44h_1h_1h_0_6)
publickey_44h_1h_1h_0_6=$(bx hd-to-public $hd_m_44h_1h_1h_0_6 | bx hd-to-ec)
publickeyhash_44h_1h_1h_0_6=$(bx sha256 $publickey_44h_1h_1h_0_6 | bx ripemd160)
address_44h_1h_1h_0_6=$(bx hd-to-public $hd_m_44h_1h_1h_0_6 | bx hd-to-ec | bx ec-to-address --version 111)

echo $address_44h_1h_1h_0_6
echo $publickey_44h_1h_1h_0_6
echo $publickeyhash_44h_1h_1h_0_6

### Fetch previous UTXO's Alice is spending.

In [ ]:
# bx fetch-tx [transaction_hash] -f json | jq ".transaction.outputs[output_index]"



In [ ]:
# previous_txid=
# previous_output_index=
# previous_output_amount=



<hr style="border: 0.5px dashed #000;">

## 1) Opening the payment channel
<br>
<img src="images/payment_channel_funding.jpg" alt="drawing" style="" width="700px"/>

## Alice creates the funding transaction

### A + B multisig output script
* `2`
* `public key A`
* `public key B`
* `2`
* `checkmultisig`


In [ ]:
# bx script-encode "2 [pubkey_alice] [pubkey_bob] 2 checkmultisig"

# funding_output_script=


### Calculate output amount

* `100 Satoshi/sigops + 1 Satoshi/Byte` 
* Non-segwit multisig counts as `4 * 20 sigops`


In [ ]:
# tx bytes: 4 + 1 + 1*(32+4+1+(1+72+1+33)+4) + 1 + 1*(8+1+(1+34+34+1+1)) + 4 

# tx_byte_count=


In [ ]:
# Output amount total.

# funding_output_amount=


### Encode transaction for signing


In [ ]:
# bx tx-encode \
# --input [previous tx id]:[index]:[sequence] \
# --output [output script]:[output amount]

# funding_tx=


### Alice signs funding transaction.

**Fetch previous output script**

In [ ]:
# bx fetch-tx --format json [previous tx id] 

# previous_output_script=


**Sign transaction with previous output script.**

In [ ]:
# bx input-sign [private key] "previous output script" [transaction template]

# signature= 



**Set the input script into the finalised transaction.**

In [ ]:
# bx input-set "[signature] [public key point]" [transaction template]

# funding_tx=



In [ ]:
# Validate.



In [ ]:
#funding_txid=



**Important:** Alice can only broadcast the funding transaction once commitment 0 has been signed by both parties, as in the following section. Otherwise, her funds could become unspendable without Bob's cooperation. 

 ## 2) Commitment transactions `0a/0b`


Alice transfers 50000 satoshi to Bob.

### Alice's Commitment transaction `0a`



#### Output Scripts
* RSMC A
* PKPKH B

#### RSMC A
* `IF`
    * `[public key R]`
* `ELSE`
    * `[3-byte delay]`
    * `checksequenceverify`
    * `drop`
    * `[public key A]`
* `ENDIF`
* `CHECKSIG`

#### `Secret r = r_A + r_B`
* Both Alice and Bob independently generate secrets `r_A` and `r_B`
* Alice and Bob only exchange public keys `R_A` AND `R_B`
* `public key R = R_A + R_B = r_A * G + r_B * G`

#### Alice computes `public key R` by adding `R_B` to` r_A * G:`


In [ ]:
# publickey_R=



#### Encoding Output Scripts for Commitment `0a`

In [ ]:
# RSMC_A_output_script=



In [ ]:
# P2PKH_B_output_script=



#### Calculate output amount.

In [ ]:
# tx bytes: 4 + 1 + 1*(32+4+1+(1+73+73)+4) + 1 + 1*(8+1+(1+34+1+4+1+1+34+1+1)) + 1*(8+1+25) + 4 

# (32+4+1+1+72+1+33+4)

# tx_byte_count=



In [ ]:

#RSMC_A_output_amount=

#P2PKH_B_output_amount=


**Encode Commitment `0a` for signing**

In [ ]:
# bx tx-encode \
# --input [previous tx id]:[index]:[sequence] \
# --output [output script]:[output amount]

# tx_0a=



**Sign commitment transaction `0a` by Alice & Bob**

In [ ]:
# bx fetch-tx --format json [previous tx id] 

# previous_output_script=



In [ ]:
# bx input-sign [private key] "previous output script" [transaction template]

# signature_a= 
# signature_b= 



In [ ]:
# bx input-set "[signature] [public key point]" [transaction template]

# tx_0a=



In [ ]:
# tx_0a_txid=



In [ ]:
# Validate.



### Bob's commitment transaction `0b`
* Omitted here for brevity's sake: Left to student to complete.
* Follows same steps as Alice's commitment transaction `0a`

<hr style="border: 0.5px dashed #000;">


## 2) Updating the channel
<br>
<img src="images/payment_channel_update.jpg" alt="drawing" style="" width="700px"/>

Alice spends another 50000 to bob.

### A. New commitment transaction `1a`/`1b` are created
* Not shown 
* Follows previous section.


### B. Alice reveals `r_A` to Bob
* Since Alice is paying Bob, she must revoke her past commitment TX output. 
* Bob can now compute `r` = `r_A + r_B`

In [ ]:
# secret_r=




In [ ]:
# Verify computed secret r corresponds to public key R.



<hr style="border: 0.5px dashed #000;">

## 3) Alice broadcasts old state. Bob sweeps the channel.
<br>
<img src="images/payment_channel_penalty.jpg" alt="drawing" style="" width="700px"/>


### A. Alice broadcasts commitment tx `0a`

In [ ]:
# Send.



### B. Bob sweeps funds from commitment tx `0a`

#### Encode P2PKH output script

In [ ]:
# Encode output spendable by Bob

# penalty_output_script=



#### Calculate output amount

In [ ]:
# tx bytes: 4 + 1 + 1*(32+4+1+(1+73)+4) + 1*(32+4+1+(73+34)+4) + 1 + 1*(8+1+25) + 4 

# tx_byte_count=




In [ ]:
# Output amount total of both inputs being swept.

# penalty_output_amount=



#### Encode penalty transaction for signing.

In [ ]:
# bx tx-encode \
# --input [previous tx id]:[index]:[sequence] \
# --output [output script]:[output amount]

# penalty_tx=



#### Sign the transaction.


* Fetch previous `RSMC_A output script` from `tx_0a` input `0`

In [ ]:
# previous_RSMA_A_script=$(bx fetch-tx ... )



* Fetch previous `P2PKH_B output script` from `tx_0a` input `1`

In [ ]:
# previous_P2PKH_B_script=$(bx fetch-tx ... )



In [ ]:
# signature_RSMC_A=



In [ ]:
# signature_P2PKH_B=



**Set the input script into the finalised transaction.**

Input script to spend RSMC A.
* `[Sig RSMC R]`
* `1`

Input script to spend P2PKH B is
* `[Sig P2PKH B]`
* `[public key B]`


In [ ]:
# bx input-set "[signature] [public key point]" [transaction template]

# Set first input
# penalty_tx=

# Set second input
# penalty_tx=



In [ ]:
# Validate penalty tx.
